In [22]:
# import sys
# import pytz
# import os
from math import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# from astropy import coordinates
from astropy.io import fits
# # from skipper import observe#, qa
from astropy import coordinates, table
# # from descartes.patch import PolygonPatch
from shapely.geometry import Point
# from matplotlib.collections import PatchCollection
from matplotlib.patches import Circle

# from mpl_toolkits.axes_grid1 import make_axes_locatable
# import matplotlib as mpl

plt.style.use('mplstyle')

### Load the exposures that qualify to Merian DR1

In [8]:
db_file = 'db_merian.fits'

data = fits.getdata(db_file, 1)
db = table.Table(data).to_pandas()
db_DR1 = db[db['expnum'] < 1087049] # this only include DR1 exposures

# N708 filter
db_n708 = db_DR1[db_DR1['band']=='N708']
db_n708 = db_n708[db_n708['exptime']>300]
db_n708.loc[db_n708['racenter'] > 300, 'ra_neg'] = db_n708['racenter']-360
db_n708.loc[db_n708['racenter'] < 300, 'ra_neg'] = db_n708['racenter']

# N540 filter
db_n540 = db_DR1[db_DR1['band']=='N540']
db_n540 = db_n540[db_n540['exptime']>300]
db_n540.loc[db_n540['racenter'] > 300, 'ra_neg'] = db_n540['racenter']-360
db_n540.loc[db_n540['racenter'] < 300, 'ra_neg'] = db_n540['racenter']

### Calculate t_eff (N708 and N540)

In [9]:
transparency_0 = 1.
seeing_0 = 1.
skySB_0_N708 = 20.9
skySB_0_N540 = 22.1

transparency = db['transparency']
seeing = db['seeing']
sky = db['sky']

db_n708['t_eff'] = (db_n708['transparency']/transparency_0)**2*(db_n708['seeing']/seeing_0)**(-2)*(10**((db_n708['sky']-skySB_0_N708)/2.5))*db_n708['exptime']
db_n540['t_eff'] = (db_n540['transparency']/transparency_0)**2*(db_n540['seeing']/seeing_0)**(-2)*(10**((db_n540['sky']-skySB_0_N540)/2.5))*db_n540['exptime']

### Make a t_eff map

#### N708 Spring

In [28]:
ra_bins = np.arange(120,240,0.1)
dec_bins = np.arange(-5,6,0.1)

ra_ind = np.arange(0,len(ra_bins),1)
dec_ind = np.arange(0,len(dec_bins),1)

coords = np.array(np.meshgrid(ra_bins, dec_bins)).T.reshape(-1,2)
indx = np.array(np.meshgrid(ra_ind, dec_ind)).T.reshape(-1,2)

data = {'ra':  coords[:,0],
        'dec': coords[:,1],
        'texp': coords[:,1]-coords[:,1],
        'teff': coords[:,1]-coords[:,1],
        'incircle' : coords[:,1]-coords[:,1],
        'ra_ind': indx[:,0],
        'dec_ind': indx[:,1]
        }
df = pd.DataFrame(data)

circle_radius = (3.18/pi)**0.5

patches = []
circles_matplotlib = []
circles = [Point(db_n708['racenter'].to_numpy()[i],
                 db_n708['deccenter'].to_numpy()[i]).buffer(circle_radius) for i in range(len(db_n708))]
for i in range(len(db_n708)):
    circle = Circle((db_n708['racenter'].to_numpy()[i],db_n708['deccenter'].to_numpy()[i]),circle_radius)
    circles_matplotlib.append(circle)
    patches.append(circle)

for c in range(len(circles_matplotlib[0:-1])):
    in_or_not = circles_matplotlib[c].contains_points(coords)
    df.loc[:,'incircle'] = in_or_not
    df.loc[df['incircle']==True,'texp'] = df.loc[df['incircle']==True,'texp'] + db_n708['exptime'].to_numpy()[c]
    df.loc[df['incircle']==True,'teff'] = df.loc[df['incircle']==True,'teff'] + db_n708['t_eff'].to_numpy()[c]

    
    tot_t_eff_n708_spring = np.zeros((len(dec_bins),len(ra_bins)))


for i in range(len(df)):
    tot_t_eff_n708_spring[df['dec_ind'][i]][df['ra_ind'][i]] = df['teff'][i]


#### N540 Spring

In [33]:
ra_bins = np.arange(120,240,0.1)
dec_bins = np.arange(-5,6,0.1)

ra_ind = np.arange(0,len(ra_bins),1)
dec_ind = np.arange(0,len(dec_bins),1)

coords = np.array(np.meshgrid(ra_bins, dec_bins)).T.reshape(-1,2)
indx = np.array(np.meshgrid(ra_ind, dec_ind)).T.reshape(-1,2)

data_n540_spring = {'ra':  coords[:,0],
        'dec': coords[:,1],
        'texp': coords[:,1]-coords[:,1],
        'teff': coords[:,1]-coords[:,1],
        'incircle' : coords[:,1]-coords[:,1],
        'ra_ind': indx[:,0],
        'dec_ind': indx[:,1]
        }
df_n540_spring = pd.DataFrame(data)

circle_radius = (3.18/pi)**0.5

patches_n540_spring = []
circles_matplotlib_n540_spring = []
circles_n540_spring = [Point(db_n540['racenter'].to_numpy()[i],
                 db_n540['deccenter'].to_numpy()[i]).buffer(circle_radius) for i in range(len(db_n540))]
for i in range(len(db_n540)):
    circle = Circle((db_n540['racenter'].to_numpy()[i],db_n540['deccenter'].to_numpy()[i]),circle_radius)
    circles_matplotlib_n540_spring.append(circle)
    patches_n540_spring.append(circle)

for c in range(len(circles_matplotlib_n540_spring[0:-1])):
    in_or_not = circles_matplotlib_n540_spring[c].contains_points(coords)
    df_n540_spring.loc[:,'incircle'] = in_or_not
    df_n540_spring.loc[df_n540_spring['incircle']==True,'texp'] = df_n540_spring.loc[df_n540_spring['incircle']==True,'texp'] + db_n540['exptime'].to_numpy()[c]
    df_n540_spring.loc[df_n540_spring['incircle']==True,'teff'] = df_n540_spring.loc[df_n540_spring['incircle']==True,'teff'] + db_n540['t_eff'].to_numpy()[c]

    
    tot_t_eff_n540_spring = np.zeros((len(dec_bins),len(ra_bins)))


for i in range(len(df_n540_spring)):
    tot_t_eff_n540_spring[df_n540_spring['dec_ind'][i]][df_n540_spring['ra_ind'][i]] = df_n540_spring['teff'][i]



### Fall N708 and N540

In [34]:
# N708
ra_bins = np.arange(-40,50,0.1)
# ra_bins = ra_bins-360
dec_bins = np.arange(-7,7,0.1)
ra_ind = np.arange(0,len(ra_bins),1)
dec_ind = np.arange(0,len(dec_bins),1)
coords = np.array(np.meshgrid(ra_bins, dec_bins)).T.reshape(-1,2)
indx = np.array(np.meshgrid(ra_ind, dec_ind)).T.reshape(-1,2)

data = {'ra':  coords[:,0],
        'dec': coords[:,1],
        'texp': coords[:,1]-coords[:,1],
        'teff': coords[:,1]-coords[:,1],
        'incircle' : coords[:,1]-coords[:,1],
        'ra_ind': indx[:,0],
        'dec_ind': indx[:,1]
        }

df = pd.DataFrame(data)
patches = []
circles_matplotlib = []
circles = [Point(db_n708['ra_neg'].to_numpy()[i],
                 db_n708['deccenter'].to_numpy()[i]).buffer(circle_radius) for i in range(len(db_n708))]
for i in range(len(db_n708)):
    circle = Circle((db_n708['ra_neg'].to_numpy()[i],db_n708['deccenter'].to_numpy()[i]),circle_radius)
    circles_matplotlib.append(circle)
    patches.append(circle)

for c in range(len(circles_matplotlib[0:-1])):
    in_or_not = circles_matplotlib[c].contains_points(coords)
    df.loc[:,'incircle'] = in_or_not
    df.loc[df['incircle']==True,'texp'] = df.loc[df['incircle']==True,'texp'] + db_n708['exptime'].to_numpy()[c]
    df.loc[df['incircle']==True,'teff'] = df.loc[df['incircle']==True,'teff'] + db_n708['t_eff'].to_numpy()[c]
    tot_t_eff_n708_fall = np.zeros((len(dec_bins),len(ra_bins)))

for i in range(len(df)):
    tot_t_eff_n708_fall[df['dec_ind'][i]][df['ra_ind'][i]] = df['teff'][i]
tot_t_eff_n708_fall[tot_t_eff_n708_fall ==0] = np.nan

# N540
ra_bins = np.arange(-40,50,0.1)
# ra_bins = ra_bins-360
dec_bins = np.arange(-7,7,0.1)
ra_ind = np.arange(0,len(ra_bins),1)
dec_ind = np.arange(0,len(dec_bins),1)
coords = np.array(np.meshgrid(ra_bins, dec_bins)).T.reshape(-1,2)
indx = np.array(np.meshgrid(ra_ind, dec_ind)).T.reshape(-1,2)

data = {'ra':  coords[:,0],
        'dec': coords[:,1],
        'texp': coords[:,1]-coords[:,1],
        'teff': coords[:,1]-coords[:,1],
        'incircle' : coords[:,1]-coords[:,1],
        'ra_ind': indx[:,0],
        'dec_ind': indx[:,1]
        }
df = pd.DataFrame(data)
patches = []
circles_matplotlib = []
circles = [Point(db_n540['ra_neg'].to_numpy()[i],
                 db_n540['deccenter'].to_numpy()[i]).buffer(circle_radius) for i in range(len(db_n540))]
for i in range(len(db_n540)):
    circle = Circle((db_n540['ra_neg'].to_numpy()[i],db_n540['deccenter'].to_numpy()[i]),circle_radius)
    circles_matplotlib.append(circle)
    patches.append(circle)

for c in range(len(circles_matplotlib[0:-1])):
    in_or_not = circles_matplotlib[c].contains_points(coords)
    df.loc[:,'incircle'] = in_or_not
    df.loc[df['incircle']==True,'texp'] = df.loc[df['incircle']==True,'texp'] + db_n540['exptime'].to_numpy()[c]
    df.loc[df['incircle']==True,'teff'] = df.loc[df['incircle']==True,'teff'] + db_n540['t_eff'].to_numpy()[c]
    tot_t_eff_n540_fall = np.zeros((len(dec_bins),len(ra_bins)))

for i in range(len(df)):
    tot_t_eff_n540_fall[df['dec_ind'][i]][df['ra_ind'][i]] = df['teff'][i]
tot_t_eff_n540_fall[tot_t_eff_n540_fall ==0] = np.nan
